## Libraries

#### Python Integrated or in Folder

In [1]:
import ipywidgets as widgets
import thinkdsp as thinkdsp
from thinkdsp import play_wave
from thinkdsp import read_wave

#### Require Installation

##### Enter these commands into your terminal:
    pip install soundfile
    pip install pyloudnorm
    pip install pydub

## Variables

#### NOT Recommended for change

In [2]:
Freqspecturmf0 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Displayed Frequency on EQ 
Freqspecturmfl = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Lower Frequency for EQ
Freqspecturmfh = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Higher Frequency for EQ

In [3]:
Bands = 10 # Standard Band Count
Controllrange = [-12, 12, 0] # [Decrease, Gain, Default] in dB
EQBandmin = 5
EQBandmax = 20
EQstep = 5

In [ ]:
# Generates Grid for EQ
GraphicEQ = widgets.GridspecLayout(Bands, 1)

#### Recommended for change

In [4]:
Text1 = widgets.Output()
with Text1:
    print("Choose your File, that you want to edit:")
    
Text2 = widgets.Output()
with Text2:
    print("\nChoose the focus of the EQ bandpass:")
    
Text3 = widgets.Output()
with Text3:
    print("\nChoose the EQ Slider Count:")
    
Text4 = widgets.Output()
with Text4:
    print("\nEqualizer:")

In [5]:
UploadFileName = "Upload"
FileFormat = ".wav"

## Functions

##### Calculating Base Frequency of choosen Type of EQ 

In [6]:
def returnbasefreq(SetFilterType, ChooseBandCount):
    # Scans input of choosen Type of EQ and gives back predefined start Frequencies for an 8 Band EQ
    if (SetFilterType == 'Bass'):
        f_base = 20
    if (SetFilterType == 'Mid'):
        f_base = 40
    if (SetFilterType == 'High'):
        f_base = 80
    
    # Modulates Frequency depending on difference between 8 / ChoosenBand
    f_base = f_base * (8/ChooseBandCount)
        
    return f_base

##### Calculates Base Frequencies that are displayed on EQ

In [7]:
def calculate_frequencies(SetFilterType, ChooseBandCount):
    #resets values in Freqspecturm
    Freqspecturmf0 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Clears all values that were precalculatet
    
    # Select start frequency for calculation
    f_base = int(returnbasefreq(SetFilterType, ChooseBandCount))
    # Calculates Modulation Rate for n-Frequencies
    f_mod = pow(2, 10 / ChooseBandCount)
    
    # Generates Freqspecturm for given Type and Bandcount
    Freqspecturmf0[0] = f_base
    
    for i in range(1, ChooseBandCount+1):
        Freqspecturmf0[i] = int(f_mod * Freqspecturmf0[i-1]) # int() Rounds Value to non decimal
    
    # print (Freqspecturm)
    return Freqspecturmf0

# SetFilterType = 'Mid'
# ChooseBandCount = 20
# calculate_frequencies(SetFilterType, ChooseBandCount)

##### Calculates log average of f0[n] and f0[n-1]

In [8]:
def calculate_frequencieslow(Freqspecturmf0, ChooseBandCount):
    # For the value of fl[0] of f0[0] we need to calculate f0[-1], fortunatly because we dont need a highpass on the first EQ we can set fl[0] = 0
    Freqspecturmfl = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Clears all values that were precalculatet
    
    Freqspecturmfl[0] = 0
    
    for i in range(1, ChooseBandCount):
            Freqspecturmfl[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i-1]))
    
    return Freqspecturmfl

##### Calculates log average of f0[n] and f0[n+1]

In [9]:
def calculate_frequencieshigh(Freqspecturmf0, ChooseBandCount):
    # For the value of fh[n] of f0[n] we need to calculate f0[n+1]
    Freqspecturmfh = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # Clears all values that were precalculatet
    
    f_mod = pow(2, 10 / ChooseBandCount)
    fnp1 = f_mod * Freqspecturmf0[ChooseBandCount]
    
    for i in range(0, ChooseBandCount):
            Freqspecturmfh[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i+1]))
    
    return Freqspecturmfh

##### Normalising Audio

In [10]:
# Value is the dB Value that the file will be normalised with
# void function: Overrides [FileName] File

def Normalise_Audio (FileName, FileFormat, Normalise):
    # Load Audio File as shape(data, wave)
    data, rate = sf.read(FileName + FileFormat)

    # Initiolising loudness Meter by BS.1770 standart 
    meter = pyln.Meter(rate) 
    # measure loudness in dB
    loudness = meter.integrated_loudness(data)
    
    # loudness normalize audio to [Normalise] dB LUFS
    loudness_normalized_audio = pyln.normalize.loudness(data, loudness, Normalise)
    sf.write(FileName  + FileFormat, loudness_normalized_audio, rate)

##### Generating temporary files

In [11]:
# Generates Temp Copies for Processing every single Band
# File Name is the UploadFile Name
# FileName must not contail ".wav" !!!

def Generate_Copies(FileName, FileFormat, BandpassCount):
    for i in range(0, ChooseBandCount-1):
        FileName.export(FinishedFileName + string(ChooseBandCount) + FileFormat, "wav")

##### Bandpass Filter 

In [12]:
# Overwrites [n] temp Audio File
# Filename is the current edited Bandpass
# Gain is the current used Gain value of said bandpass

def Bandpass_Filter (FileName, FileFormat, n, Gain, fl, fh):
    # Load Audio File
    wave = read_wave(FileName + string(n) + FileFormat)
    # Lowpass filter with fh
    spectrum.low_pass(fh)
    # Highpass filter with fl
    spectrum.high_pass(fl)
    # Overrides Temp Audio File
    with open(FileName + string(n) + FileFormat, "w+b") as edit:
        edit.write(wave)

##### Build Finished Audio

In [13]:
# Overlaps several AudioFiles which have the same base name + and adressing Number + ".wav"
# Example "FilterTemp" + string(n) + ".wav" starting from 0 to Bandpasscount-1
# FileName must not contail ".wav" !!!
# FileType is the file ending, for example ".wav"
# FinishedFileName is the Filename which the Audio will be saved. Must contain Filename+".wav"

def Build_Audio (FileName, FileFormat, FinishedFileName, BandpassCount):
    # Load first Adio file
    fullaudio = AudioSegment.from_wav(FileName + "0" + FileType)
    # Generates loop
    for i in range(0, ChooseBandCount-1):
        # import current file
        temp = AudioSegment.from_wav(FileName + string(ChooseBandCount) + FileFormat)
        # Overlays fullaudio with new generated wave file
        fullaudio = fullaudio.overlay(temp)

    # Saves fullaudio
    fullaudio.export(FinishedFileName, FileFormat)
    
    # Deletes temp Files (optional)

## Main Calculation

## User Interface

## Formulas Used

## Sources

#### Example Audio File Credits
    Song: Dimension 
    Creator: Creo 
    Youtube: https://www.youtube.com/channel/UCsCWA3Y3JppL6feQiMRgm6Q 
    Website: https://creo-music.com/track/dimension
    Licensed under: https://creativecommons.org/licenses/by/4.0/

#### Programming
    https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
    https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    https://en.wikibooks.org/wiki/LaTeX/Mathematics

#### Formulars and Documents for Audio
    https://www.teachmeaudio.com/mixing/techniques/audio-spectrum#upper-midrange
    https://sound.stackexchange.com/questions/14101/what-is-the-frequency-step-formula-for-10-and-31-band-eqs
    http://www.sengpielaudio.com/calculator-octave.html
    https://github.com/AllenDowney/ThinkDSP

#### Other Programms used
    Music editing: Audacity
    Music Download: https://github.com/yt-dlp/yt-dlp

#### Special Thanks
    https://www.senarclens.eu/~gerald/

## Programmer

#### Programmed by Thomas Pail
    GitHub: https://github.com/Tomaru-Pai